# 100daysofML

## Titanic: Machine Learning from Disaster 

### Resumo


teste teste teste

teste test 2 

### Importanto bibliotecas utilizadas 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm as cm #colormap
import seaborn as sns

% matplotlib inline


### Abrindo dataset 

In [4]:
#Abrindo dataset com ID passagens e sobrevivencia, onde 1 = sobreviveu e 0 = não sobreviveu
#gender_submission = pd.read_csv("gender_submission.csv")

#Se eu quero filtrar uma coluna, devo colocar como a coluna index e depois filtrar usando a função filter do pandas como abaixo
#gender_submission = pd.read_csv("gender_submission.csv", index_col = "Survived")
#gender_submission.filter(like='0', axis=0)
        
#Abrindo do dataset de treino do modelo que contem PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
treino = pd.read_csv("train.csv")
teste = pd.read_csv("test.csv")

treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Informações dos datasets

In [ ]:
treino.info()
print("----------------------------------")
teste.info()

#### Descrição do dataset 
1. Do nosso dataset de treino: 
    - 891 entradas (passageiros)
    - 12 colunas, sendo que dessas colunas 7 são numéricas


2. Do nosso dataset de teste
    - 414 (passageiros)
    - 11 colunas, sendo que 6 são numéricas

In [ ]:
#Descrição das variaveis 

variaveis_info = pd.DataFrame({"Variáves": ["PassengerId", 'Survived', 'Pclass', 'Name', 'Sex', 'Age','SibSp', 'Parch', 'Ticket', 'Fare',' Cabin', 'Embarked'
], "Definição":["Identificação do passageiros", "Sobrevivência", "Classe da passagem", 'Nome', 'Sexo', 'Idade', 'Irmão/conjuges a bordo', 'dos pais/filhos a bordo', 'Numero da passagem', 'Valor da passagem', 'Numero da cabine', 'Portão de embarque'],"Chaves":[" "," 1 = Sobreviveu; 0 = Não sobreviveu", " 1 = 1st, 2 = 2nd, 3 = 3rd", " ", " ", " ", " ", " ", " ", " ", " ", "C = Cherbourg, Q = Queenstown, S = Southampton"]})
variaveis_info

### Limpeza dos dados usando os 4 C's 
     - Corrigindo  - Se há outliers 
     - Complementando - Se há valores null 
     - Criando - Feature engineenging 
     - Convertendo - dados de diferente tipo (int, float, object...)

In [ ]:
print('Treino com valores vazios:\n', treino.isnull().sum()) 
print("----------------------------")
print('Teste com valores vazios:\n',teste.isnull().sum())

Valores como Age, Cabin e Embarked  no dataset de treino estão vazios 

Valores como Age, Cabin e Fare no dataset de teste estão vazios

In [ ]:
#Dados estatisticos do treino 
treino.describe()

In [ ]:
#Dados estatisticos do teste 
teste.describe()

In [ ]:
treino.head()

### Preencher valores vazios e remover colunas que creio que não seja util  

In [ ]:
limpeza_dados = [treino, teste]

for dataset in limpeza_dados:    
    #preencher com a media dos valores
    dataset['Age'].fillna(dataset['Age'].mean(), inplace = True)

    #preencher embarked com a moda
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)

    #preencher fare with mediana
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    
#deletando colunas de passengerId, Cabin e Ticket 
drop_column = ['PassengerId','Cabin', 'Ticket']
treino.drop(drop_column, axis = 1, inplace = True)

print(treino.isnull().sum()) 
print("----------------------------")
print(teste.isnull().sum())


## Feature Engineering 


In [ ]:
#Apartir das Features SibSp e Parch, foi criado uma nova feature do tamanho da familia  
for dataset in limpeza_dados:
    dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"] + 1
    
    #Verificar se é um passageiro sozinho ou com familia 
    dataset["Alone"] = 1 #onde 1 = sim 
    dataset["Alone"].loc[dataset["FamilySize"] > 1] = 0 # onde 0 = não 
    
    dataset["Title"] = dataset["Name"].str.split(", ", expand = True)[1].str.split(".", expand = True)[0] #separando o titulo Mrs, miss, mister, etc e criando ua nova coluna 
    

print(treino['Title'].value_counts())
print("-"*10) 


treino.info()
teste.info()


## Converter valores categorico

In [ ]:
from sklearn import preprocessing

label = preprocessing.LabelEncoder()

for dataset in limpeza_dados:
    dataset["Sex_Encode"] = label.fit_transform(dataset["Sex"])
    dataset["Embarked_Encode"] = label.fit_transform(dataset["Embarked"])
    dataset["Title_Encode"] = label.fit_transform(dataset["Title"])

### Verificando se os dados estão "normalizados"

In [ ]:
print("Treino com valores vazios: \n", treino.isnull().sum())
print("-"*10)
print (treino.info())
print("-"*10)

print('Teste com valores vazios:\n', teste.isnull().sum())
print("-"*10)
print (teste.info())
print("-"*10)   

In [ ]:
treino[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
treino[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

g = sns.FacetGrid(treino, col='Survived')
g.map(plt.hist, 'Pclass', bins=20)

In [ ]:
#Agrupamento de sobrevivente por localização de embarque
treino[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
#histograma de sobreviventes por idade 

g = sns.FacetGrid(treino, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
#heatmap 

fig = plt.figure(figsize = (15,8))

ax1 = fig.add_subplot(111)
cmap = cm.get_cmap('Greys', 30)
cax = ax1.imshow(treino.corr(), interpolation="nearest", cmap=cmap)
plt.title("Matrix de correlação")
labels = list(treino)
ax1.set_xticklabels(labels,fontsize=10)
ax1.set_yticklabels(labels,fontsize=10)
fig.colorbar(cax)
plt.show()


#### Conclusões tiradas com os dados estatisticos 
    - Na coluna Age, é possivel observar que há missing values e a idade média é de ~30 anos, ou seja pacientes mais novos
    - Cerca de 38% dos passageiros sobreviveram 
    - A maioria das passagens era da segunda classe
    - Passageiros da classe 3 foram os que mais morreram 
    - Mulheres sobreviveram mais que homens 
    

In [ ]:
#Dados das colunas de strings 
treino.describe(include=['O'])


#### Conclusões tiradas 
    - As colunas Cabin e Embarked há missing values 
    - Cerca de 65% dos passageiros são do sexo masculino 
    - Apesar de não possuir 3 valores é possivel falar que a maioria dos passageiros vem de Southampton
    - Alguns passageiros dividem cabines, poso dizer isso pois há valores duplicados
    - há certa de 23% de tickets duplicados
   

### Decisões que podem ser tomadas 
    - Considerar Classe, Idade, Sexo para o modelo 
    
#### Então vamos explorar mais esses dados

In [ ]:
#Analisando Classe juntamente com sobrevivencia e idade, pois observei que há uma variação entre essas variaveis  

grid = sns.FacetGrid(treino, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

### Conclusões tiradas 
    - Na Classe 1 
        - como já pontuei anteriormente passageiros dessa classe sobreviveram mais que morreram 
        - a distribuição de idade é abrangente, então não houve nessa classe em especifico uma prioridade entre as idades dos passageiros 
    - Na Classe 2 
        - Nessa classe passageiros entre 20 e 40 anos foram os que mais morreram 
    - Na Classe 3
        - Foi a classe que mais tinha passageiros, e que mais morreram sendo a idade entre 20 e 40 anos predominante 
        - Aqui podemos observar que passageiros jovens tiveram prioridade em que quesito de sobrevivencia    

In [ ]:
# Agora vamos explorar Sexo, Embarque, classe e sobrevivencia 

grid = sns.FacetGrid(treino, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()
